# Step 1: Install and Load Libraries

In [3]:
# Install libraries if not already installed
#!pip install pandas nltk
#!pip install scikit-learn

# Import necessary libraries
import pandas as pd
import nltk
import re
import math
from collections import defaultdict
from tabulate import tabulate


# Download NLTK stopwords and punkt tokenizer
#nltk.download("stopwords")
#nltk.download("punkt")

# Initialize stop words and stemmer
stop_words = set(nltk.corpus.stopwords.words("english"))
stemmer = nltk.PorterStemmer()

# Load the CSV dataset
dataset_path = "wiki_movie_plots_deduped.csv"
movies_df = pd.read_csv(dataset_path)

# Display the first few rows to verify the dataset
#movies_df.head()


# Step 10.1 Load CISI 

In [5]:
def load_cisi_dataset():
    # File paths (modify based on file locations)
    cisi_docs_path = "CISI.ALL"  # Contains documents
    cisi_queries_path = "CISI.QRY"  # Contains queries
    cisi_rels_path = "CISI.REL"  # Contains relevance judgments
    
    # Load documents
    documents = {}
    with open(cisi_docs_path, "r") as file:
        raw_data = file.read().split(".I")  # Split by the document section header
        for entry in raw_data[1:]:  # Skip the first entry, as it's empty or unnecessary
            lines = entry.strip().split("\n")
            # Ensure there's an ID to extract, skip over anything that doesn't look like an ID
            try:
                doc_id = int(lines[0].strip())  # Get the first line as the document ID
            except ValueError:
                continue  # Skip invalid lines
            content = "\n".join(lines[1:]).split(".X")[0].strip()  # Extract document content up to .X
            documents[doc_id] = content

    # Load queries
    queries = {}
    with open(cisi_queries_path, "r") as file:
        raw_data = file.read().split(".I")
        for entry in raw_data[1:]:
            lines = entry.strip().split("\n")
            # Handle cases where the first line might not be a valid integer ID
            try:
                query_id = int(lines[0].strip())
            except ValueError:
                continue  # Skip invalid lines
            content = "\n".join(lines[1:]).split(".W")[1].strip()  # Query content after ".W"
            queries[query_id] = content

    # Load relevance judgments
    relevances = {}
    with open(cisi_rels_path, "r") as file:
        for line in file:
            # Handle lines with mixed data, non-integer values, or irrelevant rows
            parts = line.strip().split()
            if len(parts) < 2:  # Skip lines with fewer than 2 entries (invalid entries)
                continue
            try:
                query_id = int(parts[0])
                doc_id = int(parts[1])  # Only convert the first two parts
            except ValueError:
                continue  # Skip any lines with invalid number conversions
            relevances.setdefault(query_id, []).append(doc_id)

    return documents, queries, relevances

# Load the dataset
documents, queries, relevances = load_cisi_dataset()

# Check the loaded data
print("Loaded dataset:")
print(f"- Number of documents: {len(documents)}")
print(f"- Number of queries: {len(queries)}")
print(f"- Number of queries with relevance data: {len(relevances)}")


Loaded dataset:
- Number of documents: 1460
- Number of queries: 112
- Number of queries with relevance data: 76


# step 10.2 Preprocess CISI

In [9]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import json

# Preprocessing function
def preprocess_text(text):
    # Lowercase and remove special characters
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", "", text)  # Remove anything that's not a letter or a digit
    
    # Tokenize and remove stopwords, then apply stemming
    tokens = nltk.word_tokenize(text)
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return tokens

# Process documents from CISI dataset (make sure 'documents' is a dictionary)
processed_documents = []

# Processing each document in the CISI dataset
for doc_id, doc_content in documents.items():
    try:
        # Preprocess document content
        processed_doc = preprocess_text(doc_content)
        processed_documents.append({
            "doc_id": doc_id,
            "content": processed_doc  # Store preprocessed content
        })
        
    except Exception as e:
        print(f"Error processing document {doc_id}: {e}")

# Optionally limit to the first 100 documents
processed_documents = processed_documents[:500]

# Save processed data to JSON file
with open("processed_data.json", "w") as f:
    json.dump(processed_documents, f)

# Checking the saved output
#print(f"Processed data for {len(processed_documents)} documents.")

# Display first few processed documents
#print(processed_documents[:2])  # Display first processed document

# Step 2: Save document on Json 

In [11]:
# Process all movie titles and plots from the dataset
def get_all_movie_plots(df):
    documents = []
    for _, row in df.iterrows():
        documents.append({
            "title": row['Title'],
            "content": row['Plot']
        })
    return documents

# Fetch all movie plots from the dataset
documents = get_all_movie_plots(movies_df)

# Save the processed documents to a JSON file
with open("all_movie_plots_data.json", "w") as f:
    json.dump(documents, f)

# Display a sample of the documents to verify content
#documents[:1]  # Show the first document as an example


# Step 3: Processes the documents

In [13]:
# Preprocessing function
def preprocess_text(text):
    # Lowercase and remove special characters
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", "", text)
    
    # Tokenize, remove stop words, and apply stemming
    tokens = nltk.word_tokenize(text)
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return tokens

# Limit processing to the first 200 documents
processed_documents = []

for i, doc in enumerate(documents[:200]):  # Only process the first 200 articles
    try:
        processed_documents.append({
            "title": doc["title"],
            "content": preprocess_text(doc["content"])
        })
        
    except Exception as e:
        print(f"Error processing document {i + 1}: {e}")
       
# Save processed documents incrementally
with open("processed_data.json", "w") as f:
    json.dump(processed_documents, f)

# Display a sample to verify
#processed_documents[:1]  # Show the first processed document as an example

# Step 3.Α: Processes the documents (Includes stopwords)

In [16]:
# Preprocessing function: Includes stopwords
def preprocess_text(text):
    # Lowercase and remove special characters
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", "", text)
    
    # Tokenize and retain stopwords
    tokens = nltk.word_tokenize(text)
    return tokens

# Limit processing to the first 200 documents
processed_documents = []

for i, doc in enumerate(documents[:200]):  # Only process the first 200 articles
    try:
        processed_documents.append({
            "title": doc["title"],
            "content": preprocess_text(doc["content"])
        })
        
    except Exception as e:
        print(f"Error processing document {i + 1}: {e}")
       
# Save processed documents incrementally
with open("processed_data.json", "w") as f:
    json.dump(processed_documents, f)

# Display a sample to verify
#processed_documents[:1]  # Show the first processed document as an example


# Step 3.Β: Processes the documents(Skips stemming)

In [19]:
# Preprocessing function: Skips stemming
def preprocess_text(text):
    # Lowercase and remove special characters
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", "", text)
    
    # Tokenize and remove stopwords, without stemming
    tokens = nltk.word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

# Limit processing to the first 200 documents
processed_documents = []

for i, doc in enumerate(documents[:200]):  # Only process the first 200 articles
    try:
        processed_documents.append({
            "title": doc["title"],
            "content": preprocess_text(doc["content"])
        })
        
    except Exception as e:
        print(f"Error processing document {i + 1}: {e}")
       
# Save processed documents incrementally
with open("processed_data.json", "w") as f:
    json.dump(processed_documents, f)

# Display a sample to verify
#processed_documents[:1]  # Show the first processed document as an example

# Step 4: Create inverted index

In [22]:
# Initialize an empty inverted index
inverted_index = defaultdict(dict)

# Populate the inverted index with term frequencies
for doc_id, doc in enumerate(processed_documents):
    for term in doc["content"]:
        if doc_id in inverted_index[term]:
            inverted_index[term][doc_id] += 1
        else:
            inverted_index[term][doc_id] = 1

# Save the inverted index to a JSON file
with open("inverted_index.json", "w") as f:
    json.dump(inverted_index, f)

# Display a small part of the inverted index to verify content
#dict(list(inverted_index.items())[:5])  # Show the first 5 terms in the index


# Step 5:Boolean Retrieval

In [25]:
# Parse the query with Boolean operators
def parse_query(query):
    terms = query.lower().split()
    tokens = []
    operators = {"and", "or", "not"}
    
    for term in terms:
        if term in operators:
            tokens.append(term)
        else:
            processed_term = stemmer.stem(term) if term not in stop_words else ""
            if processed_term:
                tokens.append(processed_term)
    
    return tokens

# Boolean retrieval function
def boolean_retrieval(query_tokens):
    result_set = set(range(len(processed_documents)))  # Start with all documents
    current_operation = "and"
    
    for token in query_tokens:
        if token in {"and", "or", "not"}:
            current_operation = token
        else:
            matching_docs = set(inverted_index.get(token, {}).keys())
            if current_operation == "and":
                result_set &= matching_docs
            elif current_operation == "or":
                result_set |= matching_docs
            elif current_operation == "not":
                result_set -= matching_docs
    
    return list(result_set)

# Step 6:TF-IDF Ranking

In [28]:
# Compute TF-IDF weight for a term in a document
def compute_tf_idf(term, doc_id):
    term_frequency = inverted_index[term].get(doc_id, 0)
    if term_frequency == 0:
        return 0
    document_frequency = len(inverted_index[term])
    inverse_document_frequency = math.log(len(processed_documents) / (1 + document_frequency))
    return term_frequency * inverse_document_frequency

# Rank results by TF-IDF scores
def rank_results_tf_idf(query_tokens, result_docs):
    doc_scores = {}
    for doc_id in result_docs:
        score = 0
        for term in query_tokens:
            if term not in {"and", "or", "not"}:
                score += compute_tf_idf(term, doc_id)
        doc_scores[doc_id] = score
    return sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)


# Step 7:BM25 Ranking

In [31]:
# BM25 parameters
k1 = 1.5  # Term frequency saturation parameter
b = 0.75  # Length normalization parameter

# Precompute document lengths and average length
document_lengths = [len(doc["content"]) for doc in processed_documents]
avg_doc_length = sum(document_lengths) / len(document_lengths)

# Function to compute BM25 score for a term in a document
def compute_bm25(term, doc_id):
    term_frequency = inverted_index[term].get(doc_id, 0)
    document_frequency = len(inverted_index[term])
    N = len(processed_documents)
    
    # Calculate IDF component with BM25 modification
    idf = math.log((N - document_frequency + 0.5) / (document_frequency + 0.5) + 1)
    
    # Calculate the BM25 term score
    doc_length = document_lengths[doc_id]
    tf_component = (term_frequency * (k1 + 1)) / (term_frequency + k1 * (1 - b + b * (doc_length / avg_doc_length)))
    
    return idf * tf_component

# Rank results by BM25 scores
def rank_results_bm25(query_tokens, result_docs):
    doc_scores = {}
    for doc_id in result_docs:
        score = 0
        for term in query_tokens:
            if term not in {"and", "or", "not"}:
                score += compute_bm25(term, doc_id)
        doc_scores[doc_id] = score
    return sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)


# Step 7: VSM Ranking

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def vector_space_model(query_tokens):
   
    # Filter documents using Boolean Retrieval results
    result_docs = boolean_retrieval(query_tokens)
    if not result_docs:  # No documents matched
        print("No documents match the query.")
        return []

    # Extract text for the filtered documents
    corpus = [" ".join(processed_documents[doc_id]["content"]) for doc_id in result_docs]
    query = " ".join(query_tokens)  # Represent query as a single string

    # Vectorize using TfidfVectorizer
    vectorizer = TfidfVectorizer()
    all_vectors = vectorizer.fit_transform(corpus + [query])  # Last row is the query vector

    document_vectors = all_vectors[:-1]  # Exclude query vector (last)
    query_vector = all_vectors[-1]  # Query vector

    # Compute cosine similarity using sklearn's cosine_similarity
    similarity_scores = cosine_similarity(query_vector, document_vectors).flatten()

    # Map scores to document IDs
    scored_results = [(result_docs[i], score) for i, score in enumerate(similarity_scores)]

    # Sort documents by similarity score in descending order
    sorted_results = sorted(scored_results, key=lambda x: x[1], reverse=True)

    return sorted_results

# Step 8: Search Function

In [37]:
def search_documents(query, algorithm="boolean"):
    query_tokens = preprocess_text(query)

    if algorithm == "boolean":
        # Boolean Retrieval: Display document titles
        results = boolean_retrieval(query_tokens)
        sorted_results = sorted(results)  # Sort by document ID (ascending)
        data = [[doc_id, documents[doc_id]['title']] for doc_id in sorted_results]
        print("\nBoolean Retrieval Results:")
        print(tabulate(data, headers=["Document ID", "Title"], tablefmt="grid"))
        return sorted_results

    elif algorithm == "tf-idf":
        # TF-IDF Ranking: Display document titles and scores
        result_docs = boolean_retrieval(query_tokens)
        ranked_results = rank_results_tf_idf(query_tokens, result_docs)
        # Sort by TF-IDF score (descending)
        sorted_results = sorted(ranked_results, key=lambda x: x[1], reverse=True)
        data = [[doc_id, documents[doc_id]['title'], score] for doc_id, score in sorted_results]
        print("\nTF-IDF Ranked Results:")
        print(tabulate(data, headers=["Document ID", "Title", "Score"], tablefmt="grid", floatfmt=".4f"))
        return sorted_results

    elif algorithm == "bm25":
        # BM25 Ranking: Display document titles and BM25 scores
        result_docs = boolean_retrieval(query_tokens)
        ranked_results = rank_results_bm25(query_tokens, result_docs)
        # Sort by BM25 score (descending)
        sorted_results = sorted(ranked_results, key=lambda x: x[1], reverse=True)
        data = [[doc_id, documents[doc_id]['title'], score] for doc_id, score in sorted_results]
        print("\nOkapi BM25 Ranked Results:")
        print(tabulate(data, headers=["Document ID", "Title", "BM25 Score"], tablefmt="grid", floatfmt=".4f"))
        return sorted_results

    elif algorithm == "vsm":
        # VSM Ranking: Display document titles and similarity scores
        ranked_results = vector_space_model(query_tokens)
        # Already sorted by VSM score (descending) within `vector_space_model`
        data = [[doc_id, documents[doc_id]['title'], score] for doc_id, score in ranked_results[:10]]  # Top 10 results
        print("\nVSM Ranked Results:")
        print(tabulate(data, headers=["Document ID", "Title", "VSM Score"], tablefmt="grid", floatfmt=".4f"))
        return ranked_results

    else:
        print("Unsupported algorithm selected.")
        return []


# Search Fuction for CISI

In [107]:
def search_documents(query, algorithm="boolean"):
    query_tokens = preprocess_text(query)

    if algorithm == "boolean":
        # Boolean Retrieval: Display document titles (or content in case of CISI)
        results = boolean_retrieval(query_tokens)
        sorted_results = sorted(results)  # Sort by document ID (ascending)

        data = []
        for doc_id in sorted_results:
            if doc_id in documents:
                if isinstance(documents[doc_id], dict) and 'title' in documents[doc_id]:
                    title = documents[doc_id]['title']
                else:
                    # Handle CISI dataset (simple document content as title)
                    title = documents[doc_id][:100]  # First 100 characters or a preview
                data.append([doc_id, title])

        print("\nBoolean Retrieval Results:")
        print(tabulate(data, headers=["Document ID", "Title"], tablefmt="grid"))
        return sorted_results

    elif algorithm == "tf-idf":
        # TF-IDF Ranking: Display document titles and scores
        result_docs = boolean_retrieval(query_tokens)
        ranked_results = rank_results_tf_idf(query_tokens, result_docs)
        # Sort by TF-IDF score (descending)
        sorted_results = sorted(ranked_results, key=lambda x: x[1], reverse=True)
        
        data = []
        for doc_id, score in sorted_results:
            if doc_id in documents:
                if isinstance(documents[doc_id], dict) and 'title' in documents[doc_id]:
                    title = documents[doc_id]['title']
                else:
                    # Handle CISI dataset (simple document content as title)
                    title = documents[doc_id][:500]  # First 500 characters or a preview
                data.append([doc_id, title, score])

        print("\nTF-IDF Ranked Results:")
        print(tabulate(data, headers=["Document ID", "Title", "Score"], tablefmt="grid", floatfmt=".4f"))
        return sorted_results

    elif algorithm == "bm25":
        # BM25 Ranking: Display document titles and BM25 scores
        result_docs = boolean_retrieval(query_tokens)
        ranked_results = rank_results_bm25(query_tokens, result_docs)
        # Sort by BM25 score (descending)
        sorted_results = sorted(ranked_results, key=lambda x: x[1], reverse=True)

        data = []
        for doc_id, score in sorted_results:
            if doc_id in documents:
                if isinstance(documents[doc_id], dict) and 'title' in documents[doc_id]:
                    title = documents[doc_id]['title']
                else:
                    # Handle CISI dataset (simple document content as title)
                    title = documents[doc_id][:500]  # First 500 characters or a preview
                data.append([doc_id, title, score])

        print("\nOkapi BM25 Ranked Results:")
        print(tabulate(data, headers=["Document ID", "Title", "BM25 Score"], tablefmt="grid", floatfmt=".4f"))
        return sorted_results

    elif algorithm == "vsm":
        # VSM Ranking: Display document titles and similarity scores
        ranked_results = vector_space_model(query_tokens)
        # Already sorted by VSM score (descending) within `vector_space_model`
        
        data = []
        for doc_id, score in ranked_results[:10]:  # Top 10 results
            if doc_id in documents:
                if isinstance(documents[doc_id], dict) and 'title' in documents[doc_id]:
                    title = documents[doc_id]['title']
                else:
                    # Handle CISI dataset (simple document content as title)
                    title = documents[doc_id][:500]  # First 500 characters or a preview
                data.append([doc_id, title, score])

        print("\nVSM Ranked Results:")
        print(tabulate(data, headers=["Document ID", "Title", "VSM Score"], tablefmt="grid", floatfmt=".4f"))
        return ranked_results

    else:
        print("Unsupported algorithm selected.")
        return []


# Step 9: Interactive Query Loop

In [40]:
def main():
    while True:
        print("\n--- Document Retrieval System ---")
        print("Choose an algorithm:")
        print("1. Boolean Retrieval")
        print("2. TF-IDF")
        print("3. Okapi BM25")
        print("4. vsm")
        print("0. Exit")

        choice = input("Enter your choice (0 to exit): ").strip()

        if choice == "0":
            print("Exiting")
            break

        query = input("Enter your search query: ").strip()

        if not query:
            print("Query cannot be empty. Please try again.")
            continue

        if choice == "1":
            print("\nYou selected: Boolean Retrieval")
            search_documents(query, "boolean")
        elif choice == "2":
            print("\nYou selected: TF-IDF Ranking")
            search_documents(query, "tf-idf")
        elif choice == "3":
            print("\nYou selected: Okapi BM25")
            search_documents(query, "bm25")
        elif choice == "4":
            print("\nYou selected: Vector Space Model (VSM)")
            search_documents(query, "vsm")
        else:
            print("Invalid choice. Please enter a number between 0 and 4.")

if __name__ == "__main__":
    main()


--- Document Retrieval System ---
Choose an algorithm:
1. Boolean Retrieval
2. TF-IDF
3. Okapi BM25
4. vsm
0. Exit


Enter your choice (0 to exit):  0


Exiting


In [42]:
from sklearn.metrics import precision_score, recall_score, f1_score
from tabulate import tabulate

# Sample queries (CISI dataset)
queries = {
    "1": "What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles?",
    "2":"How can actually pertinent data, as opposed to references or entire articles themselves, be retrieved automatically in response to information requests?",
    "3":"What is information science?  Give definitions where possible."
}

# Sample relevance data 
# Format: query_id -> list of relevant document IDs (str)
relevances = {
    "1": ["28", "35", "38", "42"],  # Relevant document IDs for query 1
    "2": ["29","68","197"],
    "3": ["60", "85"]
}

# Evaluation metrics
def compute_average_precision(retrieved_docs, relevant_docs):
    num_relevant_retrieved = 0
    ap = 0.0

    for rank, doc_id in enumerate(retrieved_docs, start=1):
        if doc_id in relevant_docs:
            num_relevant_retrieved += 1
            precision_at_rank = num_relevant_retrieved / rank
            ap += precision_at_rank

    if len(relevant_docs) > 0:
        ap /= len(relevant_docs)
    
    return ap

def evaluate_search_results(retrieved_docs, query_id, relevances):
    relevant_docs = relevances[query_id]
    tp = len(set(retrieved_docs) & set(relevant_docs))  # True Positives
    fp = len(set(retrieved_docs) - set(relevant_docs))  # False Positives
    fn = len(set(relevant_docs) - set(retrieved_docs))  # False Negatives

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1

def evaluate_all_queries(queries, relevances, search_func, algorithm="boolean"):
    map_score = 0
    for query_id, query_content in queries.items():
        print(f"\nEvaluating query {query_id}: {query_content}")
        
        retrieved_docs = search_func(query_content, algorithm=algorithm)

        precision, recall, f1 = evaluate_search_results(retrieved_docs, query_id, relevances)
        ap = compute_average_precision(retrieved_docs, relevances[query_id])
        map_score += ap

        print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}, Average Precision: {ap:.4f}")

    map_score /= len(queries)
    print(f"\nMean Average Precision (MAP): {map_score:.4f}")

# Evaluation for each algorithm
print("\nEvaluating with Boolean Retrieval:")
evaluate_all_queries(queries, relevances, search_documents, algorithm="boolean")
print("\nEvaluating with TF-IDF Retrieval:")
evaluate_all_queries(queries, relevances, search_documents, algorithm="tf-idf")
print("\nEvaluating with BM25 Retrieval:")
evaluate_all_queries(queries, relevances, search_documents, algorithm="bm25")


Evaluating with Boolean Retrieval:

Evaluating query 1: What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles?

Boolean Retrieval Results:
+---------------+---------+
| Document ID   | Title   |
+===============+=========+
+---------------+---------+
Precision: 0.0000, Recall: 0.0000, F1-Score: 0.0000, Average Precision: 0.0000

Evaluating query 2: How can actually pertinent data, as opposed to references or entire articles themselves, be retrieved automatically in response to information requests?

Boolean Retrieval Results:
+---------------+---------+
| Document ID   | Title   |
+===============+=========+
+---------------+---------+
Precision: 0.0000, Recall: 0.0000, F1-Score: 0.0000, Average Precision: 0.0000

Evaluating query 3: What is information science?  Give definitions where possible.

Boolean Ret